In [1]:
from queue import PriorityQueue

class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action 
        self.path_cost = path_cost
    
    @property
    def depth(self):
        return 1 + self.parent.depth if self.parent is not None else 0
    
    def __lt__(self, other): return self.path_cost < other.path_cost

    def print_prev_states(self):
        states = [self.state]
        prev_node = self.parent
        while (prev_node):
            states = [prev_node.state] + states
            prev_node = prev_node.parent
        for i, state in enumerate(states):
            print(f'Step {i + 1}: {state}')


class PriorityQueueWithEvaluationFunction:
    def __init__(self, eval_function):
        self.queue = PriorityQueue()
        self.eval_function = eval_function
    
    def push(self, node):
        self.queue.put((self.eval_function(node), node))
    
    def pop(self):
        (_, node) = self.queue.get()
        return node
    
    def empty(self):
        return self.queue.empty()

def expand(problem, node):
    s = node.state
    child_nodes = []
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        child_nodes.append(Node(s1, node, action, cost))
    return child_nodes

**Missionary and Cannibal problem as a search problem**

In [2]:
from enum import Enum
class Side(Enum):
    LEFT = 'left'
    RIGHT = 'right'

# A class presents Missionaries and Cannibals
class MC:
    def __init__(self, m, c):
        self.m = m
        self.c = c
    def __add__(self, o): return MC(self.m + o.m, self.c + o.c)
    def __sub__(self, o): return MC(self.m - o.m, self.c - o.c)
    def __eq__(self, o) -> bool: return self.m == o.m and self.c == o.c
    def __str__(self): return f"{self.m}M,{self.c}C"
    def __repr__(self) -> str: return f"{self.m}M,{self.c}C"

# Presents the number of missionary, cannivals on both side of the river and position of the boat
class State:
    def __init__(self, left: MC, right: MC, boat: Side):
        self.left = left 
        self.right = right
        self.boat = boat

    def __hash__(self): return hash(f"l:{self.left};r:{self.right};b:{self.boat}")
    def __str__(self): return f"{self.left} ~ {self.right}; boat:{self.boat}"
    def __repr__(self) -> str: return f"{self.left} ~ {self.right}; boat:{self.boat}"
    def __eq__(self, o): return self.left == o.left and self.right == o.right and self.boat == o.boat

class MissionaryCannibalProblem():
    def __init__(self, initial_state): 
        self.initial_state = initial_state

    # Actions
    def actions(self, state: State):
        all_actions = [MC(1,1),MC(0,1), MC(0,2), MC(2,0), MC(1,0)]
        valid_actions = []
        for action in all_actions:
            if self.is_valid_action(state, action): 
                valid_actions.append(action)
        
        return valid_actions
    
    def is_goal(self, state: State):
        return state.right == MC(3,3)
    

    # Transition model
    def result(self, state: State, action: MC):
        if state.boat == Side.LEFT:
            return State(state.left - action, state.right + action, Side.RIGHT)
        else:
            return State(state.left + action, state.right - action, Side.LEFT)
    
    
    
    def action_cost(self, s, a, s1): return 1

    def is_valid_action(self, state: State, action: MC):
        if state.boat == Side.LEFT:
            resulted_state = State(
                state.left - action, state.right + action, Side.RIGHT
            )
        else:
            resulted_state = State(
                state.left + action, state.right - action, Side.LEFT
            )
        
        if resulted_state.left.m < 0 or resulted_state.left.c < 0 or resulted_state.right.m < 0 or resulted_state.right.c < 0:
            return False
        
        if resulted_state.left.m > 0 and (resulted_state.left.m < resulted_state.left.c): 
            return False
        
        if resulted_state.right.m > 0 and (resulted_state.right.m < resulted_state.right.c):
            return False
        
        return True

**Create a problem instance**

In [3]:
initial_state = State(MC(3,3), MC(0,0), Side.LEFT)
p = MissionaryCannibalProblem(initial_state)

**Best first search**

In [4]:
def best_first_graph_search(problem, evaluation_func, early_goal_test=False):
    node = Node(problem.initial_state, parent=None, action=None, path_cost=0)
    
    frontier = PriorityQueueWithEvaluationFunction(evaluation_func)
    frontier.push(node)
    reached = {problem.initial_state: node}
    
    # Early goal test for BFS
    if early_goal_test and problem.is_goal(problem.initial_state): return True, node

    while not frontier.empty():
        node = frontier.pop()
        
        if problem.is_goal(node.state): return True, node
        for child in expand(problem, node):
            s = child.state
            
            # Early goal test for BFS
            if early_goal_test and problem.is_goal(s): return True, child
           
            if not s in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.push(child)
    
    return False, None

**BFS**

In [5]:
success, bfs_solution = best_first_graph_search(
    problem=p,
    evaluation_func=lambda node: node.depth,
    early_goal_test=True
)

print("-----BFS-----")
bfs_solution.print_prev_states()

-----BFS-----
Step 1: 3M,3C ~ 0M,0C; boat:Side.LEFT
Step 2: 2M,2C ~ 1M,1C; boat:Side.RIGHT
Step 3: 3M,2C ~ 0M,1C; boat:Side.LEFT
Step 4: 3M,0C ~ 0M,3C; boat:Side.RIGHT
Step 5: 3M,1C ~ 0M,2C; boat:Side.LEFT
Step 6: 1M,1C ~ 2M,2C; boat:Side.RIGHT
Step 7: 2M,2C ~ 1M,1C; boat:Side.LEFT
Step 8: 0M,2C ~ 3M,1C; boat:Side.RIGHT
Step 9: 0M,3C ~ 3M,0C; boat:Side.LEFT
Step 10: 0M,1C ~ 3M,2C; boat:Side.RIGHT
Step 11: 0M,2C ~ 3M,1C; boat:Side.LEFT
Step 12: 0M,0C ~ 3M,3C; boat:Side.RIGHT


**DFS**

In [6]:
success, dfs_soln = best_first_graph_search(
    problem=p,
    evaluation_func=lambda node: -1 * node.depth
)
print("-----DFS-----")
dfs_soln.print_prev_states()

-----DFS-----
Step 1: 3M,3C ~ 0M,0C; boat:Side.LEFT
Step 2: 2M,2C ~ 1M,1C; boat:Side.RIGHT
Step 3: 3M,2C ~ 0M,1C; boat:Side.LEFT
Step 4: 3M,0C ~ 0M,3C; boat:Side.RIGHT
Step 5: 3M,1C ~ 0M,2C; boat:Side.LEFT
Step 6: 1M,1C ~ 2M,2C; boat:Side.RIGHT
Step 7: 2M,2C ~ 1M,1C; boat:Side.LEFT
Step 8: 0M,2C ~ 3M,1C; boat:Side.RIGHT
Step 9: 0M,3C ~ 3M,0C; boat:Side.LEFT
Step 10: 0M,1C ~ 3M,2C; boat:Side.RIGHT
Step 11: 0M,2C ~ 3M,1C; boat:Side.LEFT
Step 12: 0M,0C ~ 3M,3C; boat:Side.RIGHT


**Greedy Best First**

In [7]:
def h(node: Node):
    if(node.state.boat == Side.LEFT):
        return 2*(node.state.left.m +  node.state.left.c) - 1
    else:
        return 2*(node.state.left.m +  node.state.left.c)

success, greedy_solution = best_first_graph_search(
    problem=p,
    evaluation_func=lambda node: h(node),
)
print("----Greedy Best First----")
greedy_solution.print_prev_states()

----Greedy Best First----
Step 1: 3M,3C ~ 0M,0C; boat:Side.LEFT
Step 2: 2M,2C ~ 1M,1C; boat:Side.RIGHT
Step 3: 3M,2C ~ 0M,1C; boat:Side.LEFT
Step 4: 3M,0C ~ 0M,3C; boat:Side.RIGHT
Step 5: 3M,1C ~ 0M,2C; boat:Side.LEFT
Step 6: 1M,1C ~ 2M,2C; boat:Side.RIGHT
Step 7: 2M,2C ~ 1M,1C; boat:Side.LEFT
Step 8: 0M,2C ~ 3M,1C; boat:Side.RIGHT
Step 9: 0M,3C ~ 3M,0C; boat:Side.LEFT
Step 10: 0M,1C ~ 3M,2C; boat:Side.RIGHT
Step 11: 0M,2C ~ 3M,1C; boat:Side.LEFT
Step 12: 0M,0C ~ 3M,3C; boat:Side.RIGHT


**A star search**

In [8]:
success, a_star_solution = best_first_graph_search(
    problem=p,
    evaluation_func=lambda node: h(node) + node.path_cost
)

print("-------A*------")
a_star_solution.print_prev_states()

-------A*------
Step 1: 3M,3C ~ 0M,0C; boat:Side.LEFT
Step 2: 2M,2C ~ 1M,1C; boat:Side.RIGHT
Step 3: 3M,2C ~ 0M,1C; boat:Side.LEFT
Step 4: 3M,0C ~ 0M,3C; boat:Side.RIGHT
Step 5: 3M,1C ~ 0M,2C; boat:Side.LEFT
Step 6: 1M,1C ~ 2M,2C; boat:Side.RIGHT
Step 7: 2M,2C ~ 1M,1C; boat:Side.LEFT
Step 8: 0M,2C ~ 3M,1C; boat:Side.RIGHT
Step 9: 0M,3C ~ 3M,0C; boat:Side.LEFT
Step 10: 0M,1C ~ 3M,2C; boat:Side.RIGHT
Step 11: 0M,2C ~ 3M,1C; boat:Side.LEFT
Step 12: 0M,0C ~ 3M,3C; boat:Side.RIGHT


**DFS tree search**

In [9]:
def best_first_tree_search(problem, evaluation_func):
    node = Node(problem.initial_state, parent=None, action=None, path_cost=0)
    frontier = PriorityQueueWithEvaluationFunction(evaluation_func)
    frontier.push(node)

    while not frontier.empty():
        node = frontier.pop()
        print(node.state)
        if problem.is_goal(node.state): return True, node
        for child in expand(problem, node):
            s = child.state
            frontier.push(child)
    
    return False, None

In [10]:
dfs_solution = best_first_tree_search(p, lambda node: -1 * node.depth)

3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side.LEFT
2M,2C ~ 1M,1C; boat:Side.RIGHT
3M,3C ~ 0M,0C; boat:Side

RecursionError: maximum recursion depth exceeded